In [1]:
import random
from time import perf_counter
import datetime
import retroactive_queue

n_values = 1000000 # number of values to insert in the queue

batchs = [100, 1000, 10000, 100000, 1000000] # batch sizes to test

n_iterations = 6 # number of iterations to average the results, 4 is the most common value

In [2]:
# NOT the best way to generate random datetime objects, but we aren't measuring this, this is just random test data
def generate_random_test_data(n_elements):
    list_datetime = [0]*n_elements
    list_string = [0]*n_elements
    # generate n random datetime objects

    for i in range(n_elements):
        year = random.randint(2000, 2020)
        month = random.randint(1, 12)
        day = random.randint(1, 28)
        hour = random.randint(0, 23)
        minute = random.randint(0, 59)
        second = random.randint(0, 59)
        microsecond = random.randint(0, 999999)

        list_datetime[i] = datetime.datetime(year, month, day, hour, minute, second, microsecond)
    
    # generate n random strings
    for i in range(n_elements):
        list_string[i] = random.randint(0, 1000000)
    
    # merge so [time, value]
    for i in range(n_elements):
        list_datetime[i] = [list_datetime[i], list_string[i]]

    return list_datetime

mother_list = generate_random_test_data(1000000)

for i in range(1, 10):
    print(mother_list[random.randint(0, 1000000)])


[datetime.datetime(2000, 4, 15, 6, 10, 13, 253490), 738947]
[datetime.datetime(2009, 2, 11, 8, 44, 56, 620910), 946263]
[datetime.datetime(2014, 3, 14, 20, 13, 15, 740962), 521594]
[datetime.datetime(2010, 7, 10, 16, 57, 59, 177024), 891000]
[datetime.datetime(2000, 2, 6, 13, 2, 31, 95603), 211792]
[datetime.datetime(2001, 12, 6, 1, 53, 22, 231971), 843291]
[datetime.datetime(2012, 11, 14, 5, 40, 19, 57011), 441900]
[datetime.datetime(2016, 12, 25, 4, 28, 19, 149757), 352797]
[datetime.datetime(2010, 3, 14, 7, 56, 25, 716429), 945406]


In [3]:
def push_performance(batch_size):
    
    start = perf_counter()

    # push(random datetime, any value)
    for i in range(n_iterations): 
        queue = retroactive_queue.queue(datetime.datetime(2023, 1, 1, 12, 0, 0), "a")
        
        # push n elements
        for j in range(batch_size):
            queue.push(mother_list[j][0], mother_list[j][1])
        
        # reset queue
        queue = None
    
    end = perf_counter()
    
    return (end - start) / n_iterations

def pop_performance(batch_size):
    
    start = perf_counter()

    # push(random datetime, any value)
    for i in range(n_iterations): 
        queue = retroactive_queue.queue(datetime.datetime(2023, 1, 1, 12, 0, 0), "a")
        
        # push n elements
        for j in range(batch_size):
            queue.push(mother_list[j][0], mother_list[j][1])
        
        # pop n elements
        for j in range(batch_size):
            queue.pop(mother_list[-j][0])
        
        # reset queue
        queue = None
    
    end = perf_counter()
    
    return (end - start) / n_iterations

In [4]:
def evaluate_performance():
    push_results = []
    pop_results = []

    for batch in batchs:
        push_time = push_performance(batch)
        pop_time = pop_performance(batch)
        
        push_results.append(push_time)
        pop_results.append(pop_time)
        
        print(f"Batch size: {batch}")
        print(f"push time: {push_time}")
        print(f"pop time: {pop_time}")

    return push_results, pop_results

In [5]:
evaluate_performance()

Batch size: 100
push time: 0.0015138333352903526
pop time: 0.006299566661861415
Batch size: 1000
push time: 0.012570550005572537
pop time: 0.3892401833339439
Batch size: 10000
push time: 0.1418305666593369
pop time: 72.33401339999789


KeyboardInterrupt: 

looks like time for pop isn't as good as I thought, definitely has something to do with the inorder traversal search of the tree